In [1]:
import helper_functions as hf
from helper_functions.classes.player import Player
from helper_functions.setup.email_handling import get_email_address
import markdown
from pathlib import Path
import pandas as pd
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from bs4 import BeautifulSoup
from email import encoders



hidden_df = hf.FpathRegistry.get_hidden_responses().set_index("nickname")
df = hf.get_players().set_index("nickname", drop=False).fillna("")
df.head()

df = df.join(hidden_df[[col for col in hidden_df.columns if not col in df.columns]], how="left")

addresses = []
df = df.fillna("")
for _, player in df.iterrows():
    if any([player[f"subteam_{sport}"] != "" for sport in hf.SPORTS_LIST]):
        addresses.append(get_email_address(player["email"]))

hf.copy_to_clipboard("; ".join(addresses))
# df.loc["Unimportant Beetle"]


In [2]:

def get_email_text_for_player(player: pd.Series) -> tuple[str, str, Path]:
    email_base_text = hf.DATAPATH.joinpath("helper_texts/email_blank_text.md").read_text(encoding="utf-8")
    first_name = player["name"].split()[0]
    nickname = player["nickname"]
    team_name = player["Team"]
    cloth_color = {"Team A": "dark", "Team B": "white", "Team C": "colorful"}[team_name]
    email = get_email_address(player["email"])
    signal_link = hf.DATAPATH.joinpath("hidden/signal_entry_link.txt").read_text(encoding="utf-8")
    
    player_obj = Player.from_series(player, list(hf.ALL_MATCHES))
    schedule = player_obj.get_schedule_for_mail()
    fpath = hf.DATAPATH.joinpath(f"assets/animal_pics/full_size/{player["nickname"].lower().replace(" ", "_")}.png")
    text = email_base_text.format(first_name=first_name, nickname=nickname, schedule=schedule, team_name=team_name, cloth_color=cloth_color, signal_link=signal_link)
    html_text = markdown.markdown(text.replace("\\", "<br>"))
    return email, html_text, fpath

# player = df.loc["Thankful Kakapo"]
# address, text, fpath = get_email_text_for_player(player)
email, text, fpath = get_email_text_for_player(df.loc["Elastic Pekingese"])
# print(email, "\n", fpath.name, "\nSports week (Apr 29 - May 3): Your schedule and information")
hf.copy_to_clipboard(text)


## Sending out the emails

Following tutorial at https://realpython.com/python-send-email/ to send out mails programmatically.

In [3]:
sender_email = "@".join(["fbalzer", "mpe.mpg.de"])  # slight protection against sp am
password = input("Type your password and press enter:")
sent_emails: list[str] = ["Pushy Bulldog", "Thankful Kakapo", "Exemplary Cassowary", "Dishonest Fangtooth", "Magnificent Barracuda"]


In [10]:

def write_email_to_player(player: pd.Series, sender_email: str, password: str, force=False):
    if not force and player["nickname"] in sent_emails:
        return
    sent_emails.append(player["nickname"])
    receiver_email, text, im_path = get_email_text_for_player(player)


    html_text = markdown.markdown(text.replace("\\", "<br>"))
    soup = BeautifulSoup(html_text, 'html')
    plain_text = soup.get_text()
    # Turn these into plain/html MIMEText objects
    part1 = MIMEText(plain_text, "plain")
    part2 = MIMEText(html_text, "html")

    # Open PDF file in binary mode
    with open(im_path, "rb") as attachment:
        # Add file as application/octet-stream
        # Email client can usually download this automatically as attachment
        part3 = MIMEBase("application", "octet-stream")
        part3.set_payload(attachment.read())

    # Encode file in ASCII characters to send by email    
    encoders.encode_base64(part3)

    # Add header as key/value pair to attachment part
    part3.add_header(
        "Content-Disposition",
        f"attachment; filename= {im_path.name}",
    )

    message = MIMEMultipart("alternative")
    message["Subject"] = "Sports week (Apr 29 - May 3): Your schedule and information"
    message["From"] = sender_email
    message["To"] = receiver_email

    # Add HTML/plain-text parts to MIMEMultipart message
    # The email client will try to render the last part first
    message.attach(part3)
    message.attach(part1)
    message.attach(part2)
    printable_mail = receiver_email.split("@")[0] + "@..."
    # - Actually send the mail
    print(f"Sending mail to {printable_mail} ({player["nickname"]})...")
    port = 587
    # Create secure connection with server and send email
    context = ssl.create_default_context()
    with smtplib.SMTP("ssmtp.mpe.mpg.de", port) as server:
        server.starttls(context=context) 
        server.login("fbalzer", password)
        server.sendmail(
            sender_email, [receiver_email, sender_email], message.as_string()
        )
# for nickname in df.sort_values("response_timestamp")["nickname"]:
#     write_email_to_player(df.loc[nickname], sender_email, password, force=False)

